# Ноутбук вычисляет, какой шмот, при заданном коэффициенте загрузки, нужно надеть, чтобы максимизировать один из параметров

In [26]:
import functools
import os.path
import pickle
import typing

import pandas as pd
# noinspection PyPackageRequirements
import typing
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
# noinspection PyPackageRequirements
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '12shYYOLyU4PpdEyw5-ObvLhgRP6aggUgrIfpGA-BWIE'
CELLS_RANGE = 'A:N'
SHEET_NAMES = 'Голова Тело Руки Ноги'.split()
RANGES = ['{}!{}'.format(sheet, CELLS_RANGE) for sheet in SHEET_NAMES]


def get_list_of_lists_of_items_grouped_by_type():
    sheet = _get_sheet()
    list_of_lists_of_items_grouped_by_type = list(
        sheet.get(
            spreadsheetId=SPREADSHEET_ID,
            range=_range
        ).execute().get('values', []) for _range in RANGES
    )
    
    return list_of_lists_of_items_grouped_by_type


def find_top_combo(column_name: str):
    combos = list(_combo_gen(list(map(
        lambda item: item[1:],
        LIST_OF_LISTS_OF_ITEMS_GROUPED_BY_TYPE,
    ))))
    combo_target_sums = _calculate_combo_sums(
        combos, COLUMN_NAMES_TO_NUMBERS[column_name],
    )
    combos_mass_sums = _calculate_combo_sums(
        combos, COLUMN_NAMES_TO_NUMBERS['Вес'],
    )
    combos_to_target_sums_to_mass_sums = list(zip(
        combos, combo_target_sums, combos_mass_sums,
    ))
    limit = round(MASS_LEFT_TO_100 * MASS_COEFFICIENT, 1)
    combos_lower_than_limit = list(filter(
        lambda cfm: cfm[2] < limit,
        combos_to_target_sums_to_mass_sums,
    ))
    top_combo = max(combos_lower_than_limit, key=lambda cfm: cfm[1])
    
    return top_combo


def _get_sheet():
    """https://developers.google.com/sheets/api/quickstart/python."""
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    return sheet.values()


def _combo_gen(list_of_lists_of_items: typing.Collection):
    """
    >>> list(
    ... _combo_gen([[1,2], [3,4], [5,6,7]]))  # doctest: +NORMALIZE_WHITESPACE
    [[1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 4, 5], [1, 4, 6], [1, 4, 7],
    [2, 3, 5], [2, 3, 6], [2, 3, 7], [2, 4, 5], [2, 4, 6], [2, 4, 7]]
    """
    ranges_count = len(list_of_lists_of_items)
    ranges_lens = list(map(len, list_of_lists_of_items))
    cursors = [0 for _ in list_of_lists_of_items]

    while True:
        positions_to_ranges = zip(cursors, list_of_lists_of_items)
        combo = [
            position_to_range[1][position_to_range[0]]
            for position_to_range in positions_to_ranges
        ]
        yield combo

        top_range = -ranges_count - 1
        for range_number, rightmost_cursor in zip(
                range(-1, top_range, -1), cursors[::-1]
        ):
            rightmost_cursor += 1
            range_len = ranges_lens[range_number]

            if rightmost_cursor < range_len:
                cursors[range_number] += 1
                break
            else:
                if range_number == top_range + 1:
                    return
                cursors[range_number] = 0


def _calculate_combo_sums(combos, column_number):
    return map(
        lambda x: round(x, 2), map(
            lambda combo: functools.reduce(
                lambda acc, el: acc + float(el[column_number]), combo, 0,
            ),
            combos,
       )
    )


In [27]:
LIST_OF_LISTS_OF_ITEMS_GROUPED_BY_TYPE = (
    get_list_of_lists_of_items_grouped_by_type()
)
# noinspection PyTypeChecker
COLUMN_NAMES_TO_NUMBERS = dict(map(
    lambda num_name: reversed(num_name),
    enumerate(LIST_OF_LISTS_OF_ITEMS_GROUPED_BY_TYPE[0][0]),
))


# Сколько массы осталось до полного загруза

In [29]:
MASS_LEFT_TO_100 = 100.5 - 6 - 4

# Каков будет процент загрузки от полной

In [ ]:
MASS_COEFFICIENT = 0.25

In [30]:
top_names = COLUMN_NAMES_TO_NUMBERS.keys()[1:-1]
tops = {
    name: find_top_combo(name)
    for name in top_names
}

In [39]:
column_names = SHEET_NAMES + ['Сумма']
df_values = list(map(
    lambda item: (*[i[0] for i in item[0]], item[1]),
    tops.values(),
))
df = pd.DataFrame(
    data=df_values,
    index=top_names,
    columns=column_names,
)

# Полная таблица топовых наборов

In [38]:
df


,Голова,Тело,Руки,Ноги,Сумма
Физическая,Маска хранителя,Катаринский доспех +2,Наручи серебряного рыцаря,Алая юбка +1,136.1
Удар,Катаринский шлем,Катаринский доспех +2,Наручи разбойника,Алая юбка +1,138.9
Резкий,Шлем горгульи,Катаринский доспех +2,Наручи элитного рыцаря +1,Алая юбка +1,151.3
Выпад,Маска хранителя,Катаринский доспех +2,Наручи серебряного рыцаря,Алая юбка +1,135.6
Магия,Шлем горгульи,Алая рубаха,Наручи Хавела,Алая юбка +1,111.3
Огонь,Рваный матерчатый капюшон,Рваная матерчатая куртка,Наручи из чёрного железа,Поножи из чёрного железа,102.0
Молния,Шлем горгульи,Тусклая накидка,Наручи Хавела,Алая юбка +1,91.7
Баланс,Мешок,Доспех Хавела,Алые перчатки,Юбка полого солдата,54.0
Кровотечение,Шлем горгульи,Куртка тени,Наручи Хавела,Окровавленная юбка,117.0
Отравление,Рваный матерчатый капюшон,Рваная матерчатая куртка,Рваные матерчатые краги,Тяжёлые сапоги,152.0
